# Interactive Coin Flip Simulator

This notebook provides an interactive interface for simulating coin flip experiments using ipywidgets.

**Features:**
- Adjust number of trials with a slider or text input
- Click button to run simulations
- View histogram and statistics in real-time
- Run multiple simulations with different parameters

## Installation

Install required packages:
```bash
pip install numpy matplotlib ipywidgets
```

For Jupyter Lab, also run:
```bash
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

# Set style
plt.style.use('default')
%matplotlib inline

## Define Simulator Class

In [2]:
class CoinFlipSimulator:
    """Class to handle coin flip simulations."""
    
    def __init__(self, n_flips_per_trial=100):
        """Initialize the simulator."""
        self.n_flips_per_trial = n_flips_per_trial
        self.results = None
    
    def flip_coins(self, n_flips):
        """Simulate flipping a fair coin n_flips times."""
        flips = np.random.randint(0, 2, n_flips)
        return np.sum(flips)
    
    def run_experiment(self, n_trials):
        """Run multiple coin flipping trials."""
        self.results = np.array([self.flip_coins(self.n_flips_per_trial) 
                                 for _ in range(n_trials)])
        return self.results
    
    def get_statistics(self):
        """Calculate statistics from results."""
        if self.results is None:
            return None
        
        stats = {
            'n_trials': len(self.results),
            'n_flips': self.n_flips_per_trial,
            'mean': np.mean(self.results),
            'expected_mean': self.n_flips_per_trial / 2,
            'std_dev': np.std(self.results),
            'median': np.median(self.results),
            'min': np.min(self.results),
            'max': np.max(self.results),
            'theoretical_std': np.sqrt(self.n_flips_per_trial * 0.5 * 0.5)
        }
        return stats

# Create simulator instance
simulator = CoinFlipSimulator(n_flips_per_trial=100)
print("✓ Simulator ready!")

✓ Simulator ready!


## Create Interactive Widget

In [3]:
# Create widgets
title = widgets.HTML("<h2 style='color: #2E86AB;'>🪙 Interactive Coin Flip Simulator</h2>")

input_label = widgets.HTML("<h3 style='color: #333;'>Experiment Settings</h3>")

trials_label = widgets.HTML(
    "<b>Number of Trials:</b> (Enter number of times to flip 100 coins)"
)

# Slider for trials
trials_slider = widgets.IntSlider(
    value=1000,
    min=10,
    max=50000,
    step=10,
    description='Trials:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px')
)

# Text input for precise control
trials_text = widgets.BoundedIntText(
    value=1000,
    min=1,
    max=1000000,
    step=1,
    description='Exact value:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

# Sync slider and text
widgets.link((trials_slider, 'value'), (trials_text, 'value'))

# Run button
run_button = widgets.Button(
    description='🎲 Run Simulation',
    button_style='success',
    tooltip='Click to run the coin flip simulation',
    layout=widgets.Layout(width='300px', height='40px')
)

# Output areas
output_plot = widgets.Output(
    layout=widgets.Layout(border='1px solid #ccc', padding='10px')
)

output_stats = widgets.Output(
    layout=widgets.Layout(border='1px solid #ccc', padding='10px')
)

status_message = widgets.HTML(
    "<p style='color: #666;'>Click 'Run Simulation' to start</p>"
)

print("✓ Widgets created!")

✓ Widgets created!


## Define Update Functions

In [4]:
def update_plot():
    """Update the histogram plot."""
    output_plot.clear_output(wait=True)
    
    with output_plot:
        if simulator.results is None:
            print("Run a simulation to see the histogram")
        else:
            fig, ax = plt.subplots(figsize=(12, 6))
            
            n_flips = simulator.n_flips_per_trial
            results = simulator.results
            
            # Create histogram
            counts, bins, patches = ax.hist(results, bins=range(0, n_flips + 2), 
                                            color='steelblue', edgecolor='black', alpha=0.7)
            
            # Calculate statistics
            mean_heads = np.mean(results)
            expected_heads = n_flips / 2
            median_heads = np.median(results)
            
            # Add reference lines
            ax.axvline(mean_heads, color='red', linestyle='--', linewidth=2.5, 
                      label=f'Observed Mean: {mean_heads:.2f}')
            ax.axvline(expected_heads, color='green', linestyle='--', linewidth=2.5, 
                      label=f'Expected Mean: {expected_heads:.2f}')
            ax.axvline(median_heads, color='orange', linestyle=':', linewidth=2, 
                      label=f'Median: {median_heads:.0f}')
            
            # Labels and title
            ax.set_xlabel('Number of Heads (out of 100 flips)', fontsize=12, fontweight='bold')
            ax.set_ylabel('Frequency (Number of Trials)', fontsize=12, fontweight='bold')
            title_str = f'Coin Flip Histogram: {len(results)} Trials of {n_flips} Flips Each'
            ax.set_title(title_str, fontsize=13, fontweight='bold')
            
            # Grid
            ax.grid(True, alpha=0.3, axis='y')
            
            # Legend
            ax.legend(fontsize=11, loc='upper right')
            
            plt.tight_layout()
            plt.show()

def update_statistics():
    """Update the statistics display."""
    output_stats.clear_output(wait=True)
    
    with output_stats:
        stats = simulator.get_statistics()
        
        if stats is None:
            print("Run a simulation to see statistics")
        else:
            stats_html = f"""
            <div style='font-family: monospace; line-height: 1.6;'>
            <h4 style='color: #2E86AB;'>RESULTS SUMMARY</h4>
            <hr style='border: 1px solid #ddd;'>
            
            <b>Experiment Setup:</b><br>
            &nbsp;&nbsp;Number of Trials:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['n_trials']}<br>
            &nbsp;&nbsp;Flips per Trial:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['n_flips']}<br>
            &nbsp;&nbsp;Total Flips:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['n_trials'] * stats['n_flips']:,}<br>
            <br>
            
            <b>Observed Distribution:</b><br>
            &nbsp;&nbsp;Mean:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['mean']:.4f}<br>
            &nbsp;&nbsp;Median:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['median']:.1f}<br>
            &nbsp;&nbsp;Std Dev:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['std_dev']:.4f}<br>
            &nbsp;&nbsp;Min Heads:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['min']}<br>
            &nbsp;&nbsp;Max Heads:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['max']}<br>
            &nbsp;&nbsp;Range:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['max'] - stats['min']}<br>
            <br>
            
            <b>Theoretical Values:</b><br>
            &nbsp;&nbsp;Expected Mean:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['expected_mean']:.2f}<br>
            &nbsp;&nbsp;Theory Std Dev:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{stats['theoretical_std']:.4f}<br>
            <br>
            
            <b>Difference from Theory:</b><br>
            &nbsp;&nbsp;Mean Error:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{abs(stats['mean'] - stats['expected_mean']):.4f}<br>
            &nbsp;&nbsp;Std Dev Error:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{abs(stats['std_dev'] - stats['theoretical_std']):.4f}<br>
            <br>
            
            <b>Law of Large Numbers:</b><br>
            &nbsp;&nbsp;As you increase the number of trials,<br>
            &nbsp;&nbsp;the observed mean converges to 50.0
            </div>
            """
            display(widgets.HTML(stats_html))

def on_run_clicked(button):
    """Handle run button click."""
    try:
        n_trials = trials_text.value
        
        if n_trials <= 0:
            status_message.value = "<p style='color: red;'>Error: Number of trials must be positive</p>"
            return
        
        status_message.value = "<p style='color: blue;'>Running simulation...</p>"
        
        # Disable button during computation
        run_button.disabled = True
        
        # Run simulation
        simulator.run_experiment(n_trials)
        
        # Update display
        update_plot()
        update_statistics()
        
        # Re-enable button
        run_button.disabled = False
        status_message.value = "<p style='color: green;'>✓ Simulation complete!</p>"
        
    except Exception as e:
        status_message.value = f"<p style='color: red;'>Error: {str(e)}</p>"
        run_button.disabled = False

# Connect button to function
run_button.on_click(on_run_clicked)

print("✓ Event handlers connected!")

✓ Event handlers connected!


## Display the Interactive Interface

In [5]:
# Create layout
input_box = widgets.VBox([
    trials_label,
    trials_slider,
    trials_text,
    run_button,
    status_message
], layout=widgets.Layout(
    border='1px solid #ddd',
    padding='15px',
    width='600px'
))

stats_box = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>Statistics</h3>"),
    output_stats
], layout=widgets.Layout(
    border='1px solid #ddd',
    padding='15px',
    width='100%'
))

plot_box = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>Histogram</h3>"),
    output_plot
], layout=widgets.Layout(
    border='1px solid #ddd',
    padding='15px',
    width='100%'
))

# Main container
main_box = widgets.VBox([
    title,
    input_label,
    input_box,
    plot_box,
    stats_box
], layout=widgets.Layout(
    padding='20px',
    width='100%'
))

# Display
display(main_box)

## Instructions

1. **Adjust the number of trials** using either:
   - The slider (quick adjustment)
   - The text box (exact value)

2. **Click "🎲 Run Simulation"** to execute the experiment

3. **View the results**:
   - Histogram appears on the right
   - Statistics displayed below
   - Mean should converge to 50 as trials increase

4. **Try different values**:
   - 100 trials: Notice high variability
   - 1000 trials: Smoother distribution
   - 10000 trials: Nearly perfect bell curve

## Theory

For 100 coin flips per trial:
- **Expected mean**: 50 heads
- **Expected std dev**: √(100 × 0.5 × 0.5) ≈ 5
- **Distribution**: Binomial (approximates Normal with large n)

The Law of Large Numbers states that as you increase trials, the observed mean converges to the theoretical mean!

## Advanced: Run Your Own Experiments

Try these experiments in new cells:

In [6]:
# Example: Run multiple simulations and compare
# Uncomment and run:

# trial_counts = [100, 500, 1000, 5000]
# results_list = []

# for n_trials in trial_counts:
#     sim = CoinFlipSimulator()
#     sim.run_experiment(n_trials)
#     stats = sim.get_statistics()
#     results_list.append(stats)
#     print(f"\n{n_trials} trials: Mean = {stats['mean']:.2f}, Std Dev = {stats['std_dev']:.2f}")